In [1]:
# data_process is used to generate raw data and perform feature engineering with the data
#%run data_process
%run recommendation_engine_func.ipynb

In [2]:
import sys,os
print(sys.version)
print(os.getcwd())

3.5.2 |Anaconda 4.2.0 (x86_64)| (default, Jul  2 2016, 17:52:12) 
[GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]
/Users/zishuoli/Downloads/bigdata/final


In [3]:
'''import basic package'''
import pandas as pd
import numpy as np


'''import deep learning package'''
import keras as k
import tensorflow as tf


import requests
import json

from IPython.display import Image
from IPython.display import display
from IPython.display import HTML

import urllib.request

from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image as kimage


Using TensorFlow backend.


# Spark ALS recommendation


In [4]:
import findspark
findspark.init('/usr/local/Cellar/spark-2.2.0-bin-hadoop2.7')

from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext

from pyspark.sql.functions import sqrt

from pyspark.sql import SparkSession
from pyspark.mllib.recommendation import ALS,MatrixFactorizationModel,Rating

In [5]:
spark = SparkSession.builder.appName('final2').getOrCreate()

data = spark.read.csv('file:/Users/zishuoli/Downloads/bigdata/final/ml-latest-small/ratings.csv', inferSchema=True, header=True)

In [6]:
data.createOrReplaceTempView("test")
  
test = spark.sql('select * from test')
#test.show()

In [7]:
data = data.select('userID','movieID','rating')

In [8]:
train,test = data.randomSplit([0.7,0.3])
print(train.count())
print(test.count())
print(train.first())

test.cache()
train.cache()

70054
29950
Row(userID=1, movieID=1029, rating=3.0)


DataFrame[userID: int, movieID: int, rating: double]

In [9]:
rank = 5
numIterations = 10
model = ALS.train(train,rank,numIterations)
model.productFeatures().first()


(8,
 array('d', [-0.20483286678791046, 1.2477695941925049, 1.7491360902786255, -1.2875621318817139, 1.133804202079773]))

In [10]:
model.userFeatures().first()

(8,
 array('d', [0.8400859832763672, -0.07768601179122925, 0.6594439744949341, -0.3243723213672638, 0.2536373436450958]))

In [11]:
model.recommendUsers(196,5)

[Rating(user=410, product=196, rating=6.515625528623293),
 Rating(user=629, product=196, rating=5.130644749494991),
 Rating(user=64, product=196, rating=5.121751434768452),
 Rating(user=637, product=196, rating=4.9002622976553845),
 Rating(user=568, product=196, rating=4.893850575123901)]

In [12]:
model.predict(310,196)

1.4348849990353671

In [13]:
pred_input = train.select('userID','movieID').rdd
pred = model.predictAll(pred_input).toDF().select('rating')

true_pred = train.select('rating')

In [14]:
true_pred.show(5)

+------+
|rating|
+------+
|   3.0|
|   2.0|
|   4.0|
|   2.0|
|   3.5|
+------+
only showing top 5 rows



In [15]:
pred.show(5)

+------------------+
|            rating|
+------------------+
|1.0100425916594689|
|0.5180380794721712|
|4.4712503343485235|
| 5.120519493745328|
|3.0472310917624505|
+------------------+
only showing top 5 rows



In [16]:
mse = true_pred.subtract(pred).rdd.map(lambda x:x[0]**2).mean()
mse = np.sqrt(mse)
print(mse)

3.1024184115


In [17]:
test_input = test.select('userId','movieId').rdd
pred_test = model.predictAll(test_input).toDF().select('rating')
true_pred_test = test.select('rating')
 
mse_test = true_pred_test.subtract(pred_test).rdd.map(lambda x:x[0]**2).mean()
mse_test = np.sqrt(mse_test)
print(mse_test)

3.1024184115


# data loading & preprocess 

In [18]:
rating_matrix = np.load('rating_matrix.npy')
movieId_vs_tmdbId = np.load('movieId_vs_tmdbId.npy').item()

In [19]:
similarity_user = cosine_similarity_matrix(rating_matrix)
print('similarity user matrix shape:',similarity_user.shape)
similarity_movie = cosine_similarity_matrix(rating_matrix.T)
print('similarity movie matrix shape:',similarity_movie.shape)


similarity user matrix shape: (671, 671)
similarity movie matrix shape: (9000, 9000)


# 1. SVD Recommendation Part
## target at rating 

In [20]:
items, items_rating = svd_recommend(rating_matrix,movieId_vs_tmdbId,5)

target_id = 8
movie_list_svd = items[target_id]
url_list_svd = [get_poster(x) for x in movie_list_svd]

display_images(url_list_svd)


# 2. Cosine similarity recommendation part

## 2.1user to user similarity recommendation result

In [21]:
movie_list_user = pred_similarity_user(similarity_user,rating_matrix,movieId_vs_tmdbId,n=5)

In [22]:
target_id = 0
movie_list_user = movie_list_user[target_id]
url_list_user = [get_poster(x) for x in movie_list_user]

display_images(url_list_user)


## 2.2 item to item similarity recommendation result

In [23]:
movie_list_item = pred_similarity_item(similarity_movie,rating_matrix,movieId_vs_tmdbId,n = 5)

In [24]:
target_id = 0
movie_list_item = movie_list_item[target_id]

url_list_item = [get_poster(x) for x in movie_list_item]
display_images(url_list_item)

# 3. Deep Learning Part

In [25]:
feature_matrix = np.load('feature_matrix.npy')
movie_list_dl  = np.load('movie_list.npy')
similarity_movie_dl = cosine_similarity_matrix(feature_matrix)

In [26]:
target_id = 4
show_result(similarity_movie_dl,movieId_vs_tmdbId,movie_list_dl,target_id=target_id,n=5)

In [27]:
target_id = 666
show_result(similarity_movie_dl,movieId_vs_tmdbId,movie_list_dl,target_id=target_id,n=5)

In [28]:
target_id = 333
show_result(similarity_movie_dl,movieId_vs_tmdbId,movie_list_dl,target_id=target_id,n=5)

In [31]:
target_id = 345
show_result(similarity_movie_dl,movieId_vs_tmdbId,movie_list_dl,target_id=target_id,n=5)

In [30]:
target_id = 7
show_result(similarity_movie_dl,movieId_vs_tmdbId,movie_list_dl,target_id=target_id,n=5)